In [2]:
import asyncio
import threading
import ccxt
import configparser
import pandas as pd
import time
import warnings

from indicators import supertrend
from utilities import check_buy_sell_signals, create_synthetic_pair
from position_manager import Position

pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')

config = configparser.ConfigParser()
config.read("config.ini")


exchange = ccxt.kucoin({
  "apiKey": config['KuCoin']['apiKey'],
  "secret": config['KuCoin']['secret'],
  "password": config['KuCoin']['password']
})

In [11]:
# had to make this non-async to work in jupyter
# in main.py
# This is instead of
# async def main()
def main(base_pair=config['Bot Settings']['base_pair_default'],
         quote_pair=config['Bot Settings']['quote_pair_default']):

    running = True

    while running:
        # This code is for one timeframe. Later, do the same iterated for each timeframe.
        base_bars = exchange.fetch_ohlcv(base_pair, timeframe="1m", limit=50)
        quote_bars = exchange.fetch_ohlcv(quote_pair, timeframe="1m", limit=50)

        pair = create_synthetic_pair(base_bars, quote_bars)

        supertrend_data = supertrend(pair)

        signal = check_buy_sell_signals(supertrend_data)
        print(f"Signal: {signal}")

        if signal.higher == "LONG":
            live_position = Position().open()
            pass
        elif signal.higher == "SHORT":
            live_position = Position().open()
            pass

        # await asyncio.sleep(60)
        time.sleep(60)

# This is how the main function is called in `main.py`:
`asyncio.run(main())`
Maybe this will change later, but it's fine for now.

""" Position Class ./position_manager.py
# import statement:
# from position_manager import Position

`def open(self, exchange, synth_pair, base_pair, quote_pair, borrow_coin, borrow_qty, direction,
             order_type='market', prompt_borrow_confirmation=False):`
"""

position_example = Position().open(exchange, 'ETHUSDT/BTCUSDT', 'ETHUSD', 'BTCUSD', )